In [89]:
import requests
import asyncio
import aiohttp
import time
from bs4 import BeautifulSoup

In [90]:
# This is format of JFM search page
# https://www.cambridge.org/core/journals/journal-of-fluid-mechanics/listing?aggs%5BonlyShowAvailable%5D%5Bfilters%5D=true&pageNum=2&fts=yes&searchWithinIds=1F51BCFAA50101CAF5CB9A20F8DEA3E4

## Scrape and parse JFM search page for PDFs

Open Access only. Maybe someday I will implement institutional login.

In [91]:
url_prefix = "https://www.cambridge.org/core/journals/journal-of-fluid-mechanics/listing"
url_core = "?aggs%5BonlyShowAvailable%5D%5Bfilters%5D=true&pageNum="
url_suffix = "&fts=yes&searchWithinIds=1F51BCFAA50101CAF5CB9A20F8DEA3E4"

params = {}
pdf_links = []

# Number of pages to scrape
num_pages = 131

async def scrape(sess, url, params, page):
    async with sess.get(url=url, params=params) as response:
        contents = None
        try:
            contents = await response.text()
        except Exception as e:
            print("Unable to decode using utf8")

        if contents is None:
            contents = ''
            try:
                contents = await response.text('latin-1')
            except Exception as e:
                print("Unable to decode using latin-1")
                
    soup = BeautifulSoup(contents,'html.parser')
    links = soup.select("#maincontent div.results ul.access.links li a[data-pdf-content-id]")

    for a in links:
        pdf_links.append(format(a['href']))

async def bound_fetch(semaphore, sess, url, params, page):
    async with semaphore:
        await scrape(sess, url, params, page)

if __name__ == '__main__':
    urls = {}
    for i in range(1, num_pages+1):
        urls[i] = url_prefix + url_core + str(i) + url_suffix
        
        if i % 10 == 0:
            print("Processed 10 more pages (Total {0})".format(i))
        
    await main(urls)

Processed 10 more pages (Total 10)
Processed 10 more pages (Total 20)
Processed 10 more pages (Total 30)
Processed 10 more pages (Total 40)
Processed 10 more pages (Total 50)
Processed 10 more pages (Total 60)
Processed 10 more pages (Total 70)
Processed 10 more pages (Total 80)
Processed 10 more pages (Total 90)
Processed 10 more pages (Total 100)
Processed 10 more pages (Total 110)
Processed 10 more pages (Total 120)
Processed 10 more pages (Total 130)


### Save the list of pdf_links to file

In [95]:
with open(r'pdf_links.txt','w') as fp:
    for i,link in enumerate(pdf_links,1):
        fp.write("%s\n" % link)
        
# Instead of running the entire scraper again, we can then do:
# pdf_links = []

# with open(r'pdf_links.txt', 'r') as fp:
#     for line in fp:
#         x = line[:-1]
#         pdf_links.append(x)

## Get dates from JFM PDFs

Because they don't put the "Received" and "Revised" on the webpages. Smh

In [96]:
import re
import io

from PyPDF2 import PdfReader
from urllib.request import Request, urlopen

headers = {'User-Agent': 'Mozilla/5.0 (X11; Windows; Windows x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.5060.114 Safari/537.36'}

error_log = []
the_dates = []

def extract_date(link_part):
    
    # add the cambridge url
    cambridge = "https://www.cambridge.org"
    the_link = cambridge + link_part

    # get the pdf linked
    response = requests.get(url=the_link, headers=headers, timeout=120)
    on_fly_mem_obj = io.BytesIO(response.content)
    reader = PdfReader(on_fly_mem_obj)

    # extract text
    text = reader.pages[0].extract_text() 

    # extract received date from text
    res_search = re.search("Received", text)
    
    try:
        dates = text[int(res_search.start()):].split(")")[0]
        return dates
    except:
        error_log.append(the_link)
        return None

for i, lpart in enumerate(pdf_links, 1):
    the_dates.append(extract_date(lpart))
    if i % 10 == 0:
          print("Processed 10 more articles (Total {0})".format(i))

Processed 10 more articles (Total 10)
Processed 10 more articles (Total 20)
Processed 10 more articles (Total 30)
Processed 10 more articles (Total 40)
Processed 10 more articles (Total 50)
Processed 10 more articles (Total 60)
Processed 10 more articles (Total 70)
Processed 10 more articles (Total 80)
Processed 10 more articles (Total 90)
Processed 10 more articles (Total 100)
Processed 10 more articles (Total 110)
Processed 10 more articles (Total 120)
Processed 10 more articles (Total 130)
Processed 10 more articles (Total 140)
Processed 10 more articles (Total 150)
Processed 10 more articles (Total 160)
Processed 10 more articles (Total 170)
Processed 10 more articles (Total 180)
Processed 10 more articles (Total 190)
Processed 10 more articles (Total 200)
Processed 10 more articles (Total 210)
Processed 10 more articles (Total 220)
Processed 10 more articles (Total 230)
Processed 10 more articles (Total 240)
Processed 10 more articles (Total 250)
Processed 10 more articles (Total 

Processed 10 more articles (Total 2090)
Processed 10 more articles (Total 2100)
Processed 10 more articles (Total 2110)
Processed 10 more articles (Total 2120)
Processed 10 more articles (Total 2130)
Processed 10 more articles (Total 2140)
Processed 10 more articles (Total 2150)
Processed 10 more articles (Total 2160)
Processed 10 more articles (Total 2170)
Processed 10 more articles (Total 2180)
Processed 10 more articles (Total 2190)
Processed 10 more articles (Total 2200)
Processed 10 more articles (Total 2210)
Processed 10 more articles (Total 2220)
Processed 10 more articles (Total 2230)
Processed 10 more articles (Total 2240)
Processed 10 more articles (Total 2250)
Processed 10 more articles (Total 2260)
Processed 10 more articles (Total 2270)
Processed 10 more articles (Total 2280)
Processed 10 more articles (Total 2290)
Processed 10 more articles (Total 2300)
Processed 10 more articles (Total 2310)
Processed 10 more articles (Total 2320)
Processed 10 more articles (Total 2330)


ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [99]:
# Remove null values caused by error (e.g. cover page or errata)
the_dates = [x for x in the_dates if x is not None]

### Save dates as a text file

So that we don't have to run the date parser again (it's quite slow).

In [100]:
with open(r'dates.txt','w') as fp:
    for date in the_dates:
        fp.write("%s\n" % date)
        
# Instead of running the entire scraper again, we can then do:
# the_dates = []

# with open(r'dates.txt', 'r') as fp:
#     for line in fp:
#         x = line[:-1]
#         the_dates.append(x)

## Convert dates to database table

In [125]:
import pandas as pd

def form_date(date):
    d = [x.strip() for x in date.split(';')] 
    
    received = d[0].split(' ',1)[1]
    
    try:
        #if no revision date we assume accepted upon submission
        if len(d) == 3:
            revised = d[1].split(' ',1)[1]
            accepted = d[2].split(' ',1)[1]
        elif len(d) == 2:
            accepted = d[1].split(' ',1)[1]
            revised = accepted 
        else:
            revised = None
            accepted = None
    except:
        print(d[1], d[2])

    return [received, revised, accepted]

ddate = []
for d in the_dates:
    ddate.append(form_date(d))

df = pd.DataFrame(ddate)
df = df.apply(pd.to_datetime)
df = df.dropna()

df.columns = ["Received","Revised","Accepted"]

# Save as a csv file
# df.to_csv("dates.csv")

### Calculate mean no. of days between reception and revision

Assumption 1: Time to revise article from receiving the reviewer comments is approximately the same across all the articles in the dataset (may not be a necessarily valid assumption, but assuming everything is normally distributed, it could possibly work at least to give an estimate)

Assumption 2: Set of open-access articles is sufficiently representative of all JFM articles

In [126]:
df["Days to Revision"] = (df["Revised"] - df["Received"]).dt.days
df["Days to Acceptance"] = (df["Accepted"] - df["Received"]).dt.days

In [127]:
# Filter out only JFM articles > 2018 (past 5 years)
df_5yrs = df[(df['Received'] > "2018-01-01")]

d2r = df_5yrs.loc[:, 'Days to Revision'].mean()
d2a = df_5yrs.loc[:, 'Days to Acceptance'].mean()

print("Mean days to revision: "+str(d2r))
print("Mean days to acceptance: "+str(d2a))

Mean days to revision: 160.1948905109489
Mean days to acceptance: 195.74452554744525
